# Breadth First Search

We will be trying to find and visualize the path between Equestrian Statue of Edward VII and Bahen Center of Technology around Toronto University campus using breadth first search


__BREADTH-FIRST-SEARCH__(graph) __returns__ a route and cost, or failure  
&emsp;__if__ _graph_ first node is a goal __then return__ empty path to initial state  
&emsp;_frontier_ &larr; a **FIFO** queue initially containing one path, for the _graph_'s initial node  
&emsp;_reached_ &larr; a set of states; initially empty  
&emsp;_solution_ &larr; failure  
&emsp;__while__  _frontier_ is not empty __do__  
&emsp;&emsp;&emsp;_parent_ &larr; the first node in _frontier_  
&emsp;&emsp;&emsp;__for__ _child_ __in__ successors(_parent_) __do__   
&emsp;&emsp;&emsp;&emsp;&emsp;_s_ &larr; _child_.state  
&emsp;&emsp;&emsp;&emsp;&emsp;__if__ _s_ is a goal  __then__  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;__return__  _child_  
&emsp;&emsp;&emsp;&emsp;&emsp;__if__ _s_ is not in _reached_ __then__  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;add _s_ to _reached_  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;add _child_ to the end of _frontier_  
&emsp;__return__  _solution_

In [ ]:
%%capture

##############################################################################
# IGNORE if you are running on a local machine and have all the dependencies #
##############################################################################

# installing dependencies --- to be able to run on google colab
# it would take 2-3 minutes
!apt-get install libspatialindex-c4v5;
!pip3 install Rtree;
!pip3 install osmnx;
!pip3 install tqdm;

In [3]:
import osmnx as ox
import networkx as nx
from tqdm import tqdm
import time, random
from IPython.display import IFrame, HTML
from collections import deque
import matplotlib.pyplot as plt
from problem import *
from utils import *

In [11]:
from Utilities.utils import *

Here OSMnx find the largest connected component centered around the ```location point``` with specified ```dist``` on each side (w/n/e/s).

In [ ]:
location_point = (43.661667, -79.395)
G = ox.graph_from_point(location_point, dist=300, clean_periphery=True, simplify=True)
graph_map = ox.plot_graph_folium(G, popup_attribute='name', edge_width=2)
fig, ax = ox.plot_graph(G)

In [ ]:
# we need to save the graph 
filepath = 'data/graph.html'
graph_map.save(filepath)
IFrame(filepath, width=600, height=500)

Here you need to specify which node from our graph is the source (Equestrian Statue of Edward VII) and which is the destination node (Bahen Center of Technology). You can do so by acquiring the decimal coordinates of the desired node and use [```osmnx.distance.get_nearest_node```](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.get_nearest_node) method

I used the aforementioned method and found that the nodes for destination and source are 389677909, 55808290 respectively

In [ ]:
# marking both the source and destination node

nc = ['r' if node == 389677909 or node == 55808290 else '#336699' for node in G.nodes()]
ns = [50 if node == 389677909 or node == 55808290 else 8 for node in G.nodes()]
fig, ax = ox.plot_graph(G, node_size=ns, node_color=nc, node_zorder=2)

Each node in our graph is represented as a dictionary so manipulating them would obfuscate the algorithm manipulating them and would be unnecessary expensive computationally <b>so</b> we define class ```Node``` only retains the data we need to be able to do searching and traversing.

Please check its source code in ```./problem.py``` to know how it capture the data from the graph

In [ ]:
%%capture
source(Node)

# The Algorithm

In [ ]:
# first define the origin/source nodes as Node
origin = Node(graph = G, node = G[55808290], osmid = 55808290, distance = 0, parent = None)
destination = Node(graph = G, node = G[389677909], osmid = 389677909)

In [ ]:
bar = tqdm(total=len(G))
route = []
frontier = deque([origin])
explored = set()
found = False
while frontier and not found:
    bar.update(1); time.sleep(0.1)
    node = frontier.popleft()
    explored.add(node)
    for child in node.expand():
        if child not in explored and child not in frontier:
            if child == destination:
                route, cost = child.path()
                print(route)
                found = True
            frontier.append(child)

bar.close()
print(route)

let's see the length of that route

In [ ]:
fig, ax = ox.plot_graph_route(G, route)

In [ ]:
route_map = ox.plot_route_folium(G, route)
filepath = 'data/route.html'
route_map.save(filepath)
IFrame(filepath, width=600, height=500)

In [ ]:
G1 = ox.graph_from_xml("./export.osm", simplify = False, retain_all = True)

In [ ]:
fig, ax = ox.plot_graph(G1)

In [ ]:
nc = ['r' if node in [*find_by_osmid(G1,   295165976)]  else '#336699' for node in G1.nodes()]
ns = [50 if node in [*find_by_osmid(G1,   295165976)] else 8 for node in G1.nodes()]
fig, ax = ox.plot_graph(G1, node_size=ns, node_color=nc, node_zorder=2)

In [ ]:
def find_by_osmid(G, osmid):
    for node in G:
        for neighbour in G[node]:
            if G[node][neighbour][0]['osmid'] == osmid:
                yield neighbour

In [ ]:
[*find_by_osmid(G1,  10520720)]

In [ ]:
from ipyleaflet import Map, Marker


In [ ]:
center = ( 50.749388,7.130756 )

m = Map(center=center, zoom=15)

marker = Marker(location=(50.747603,7.136665), draggable=False)
m.add_layer(marker);
marker = Marker(location=(50.747535,7.136541), draggable=False)
m.add_layer(marker);
marker = Marker(location=(50.747111, 7.135578), draggable=False)
m.add_layer(marker);
marker = Marker(location=(50.747072,7.135416), draggable=False)
m.add_layer(marker);
marker = Marker(location=(50.746944,7.13461), draggable=False)
m.add_layer(marker);
marker = Marker(location=(50.746868,7.133971), draggable=False)
m.add_layer(marker);
marker = Marker(location=(50.74688,7.133253), draggable=False)
m.add_layer(marker);
marker = Marker(location=(50.746891,7.132963), draggable=False)
m.add_layer(marker);
marker = Marker(location=(50.746938,7.13185), draggable=False)
m.add_layer(marker);
marker = Marker(location=(50.748947, 7.131108), draggable=False)
m.add_layer(marker);
marker = Marker(location=(50.749388, 7.130756), draggable=False)
m.add_layer(marker);

m

In [ ]:
from ipyleaflet import Map, Polyline, AntPath


In [ ]:
line = AntPath(
    locations=[
        [50.747603,7.136665],
        [50.747535,7.136541],
        [50.747111, 7.135578],
        [50.747072,7.135416],
        [50.746944,7.13461],
        [50.746868,7.133971],
        [50.74688,7.133253],
        [50.746891,7.132963],
        [50.746938,7.13185],
        [50.748947, 7.131108],
        [50.749388, 7.130756]
        
    ],
#     color="green" ,
#     fill=False
    dash_array=[1, 10],
    delay=1000,
    color='#7590ba',
    pulse_color='#3f6fba'
)
m = Map(center = ( 50.749388,7.130756 ), zoom =20)
m.add_layer(line)
m

In [ ]:
G.nodes()

In [ ]:
G[50897923]

In [ ]:
G1 = ox.graph_to_gdfs(G)

In [ ]:
for row in G1[0]:
    print(G1[0][row])

In [ ]:
G1[0]

In [ ]:
x = [316884976, 3311164627, 150430583]

In [ ]:
nc = ['r' if node in x  else '#336699' for node in G.nodes()]
ns = [50 if node in x else 8 for node in G.nodes()]
fig, ax = ox.plot_graph(G, node_size=ns, node_color=nc, node_zorder=2)

In [ ]:
G[150430583]

In [ ]:
import pandas as pd

In [ ]:
for node in G1:
    for prop in node:
        print(prop)
        print((node[prop]))
        print("....................................--------------------................................")
    

In [ ]:
G[6028562355]